In [4]:
import pandas as pd
from pandasql import sqldf
import boto3
import json
import os
# python -m spacy download en
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keras import models, layers
# from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
# import numpy as np
from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'spacy'

In [3]:
with open('../boto-config.json') as json_data:
    boto_config = json.load(json_data)
    
s3 = boto3.client(
    's3',
    aws_access_key_id=boto_config['aws_access_key_id'],
    aws_secret_access_key=boto_config['aws_secret_access_key']
)

NameError: name 'json' is not defined

In [2]:
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
src_file = data_dir + '/News_Category_Dataset.json'
src_file_cleaned = data_dir + '/News_Clean.csv'

# s3.upload_file(src_file, boto_config['buckets']['kaggle'], src_file)
# s3.upload_file(src_file_cleaned, boto_config['buckets']['kaggle'], src_file_cleaned)

# s3.download_file(boto_config['buckets']['kaggle'], src_file, src_file)
# s3.download_file(boto_config['buckets']['kaggle'], src_file_cleaned, src_file_cleaned)

In [42]:
# data = pd.read_json(src_file, lines=True)
df = pd.read_csv(src_file_cleaned)
df = df[~df.clean_text.isnull()]

In [29]:
data.describe()

,authors,category,date,headline,link,short_description
count,124989,124989,124989,124989,124989,124989
unique,19250,31,1498,124560,124964,103905
top,,POLITICS,2017-01-27 00:00:00,Sunday Roundup,https://www.huffingtonpost.comhttp://highline....,
freq,14151,32739,100,90,2,19590
first,NaN,NaN,2014-04-18 00:00:00,NaN,NaN,NaN
last,NaN,NaN,2018-05-26 00:00:00,NaN,NaN,NaN


In [6]:
data.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [37]:
data.isnull().values.any()

False

In [30]:
data.columns

Index(['authors', 'category', 'date', 'headline', 'link', 'short_description'], dtype='object')

In [31]:
data.shape

(124989, 6)

In [51]:
# clean up text
nlp = spacy.load('en')

data['text'] = data['headline'] + ' ' + data['short_description']

docs = data['text'].tolist()

def token_filter(token):
    return not (token.is_punct | token.is_space | token.is_stop | len(token.text) <= 3)

filtered_tokens = []
for doc in nlp.pipe(docs):
    tokens = [token.lemma_ for token in doc if token_filter(token) and token.text not in STOP_WORDS]
    filtered_tokens.append(' '.join(tokens))

data['clean_text'] = filtered_tokens

data.to_csv(src_file_cleaned, index=False)

In [76]:
# tfidf with sklearn
tfidf = TfidfVectorizer()

bag = tfidf.fit_transform(df.loc[:, 'clean_text'])

In [124]:
lbler = LabelBinarizer()
labels = lbler.fit_transform(df['category'])
num_labels = labels.shape[1]

In [125]:
train = tfidf.transform(df.loc[:, 'clean_text'])

In [126]:
# train, test validation
X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size = 0.2, random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 1)

In [127]:
# nn with keras
model = models.Sequential()
model.add(layers.Dense(360, input_shape=(X_train.shape[1],), activation='relu'))
model.add(layers.Dense(180, activation='relu'))
model.add(layers.Dense(90, activation='relu'))
model.add(layers.Dense(num_labels, activation='softmax'))

In [128]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [129]:
history = model.fit(X_train, y_train, epochs = 10, batch_size=512, validation_data=(X_val, y_val))

Train on 89978 samples, validate on 9998 samples
Epoch 1/10
76800/89978 [========================>.....] - ETA: 14s - loss: 2.2343 - acc: 0.4123

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-129-a2e48bfdc4a5>", line 1, in <module>
    history = model.fit(X_train, y_train, epochs = 10, batch_size=512, validation_data=(X_val, y_val))
  File "/anaconda3/lib/python3.6/site-packages/keras/engine/training.py", line 1042, in fit
    validation_steps=validation_steps)
  File "/anaconda3/lib/python3.6/site-packages/keras/engine/training_arrays.py", line 199, in fit_loop
    outs = f(ins_batch)
  File "/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2661, in __call__
    return self._call(inputs)
  File "/anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py", line 2631, in _call
    fetched = self._callable_fn(*array_vals)
  File "/anaconda3/lib/python3.6/site-packages/tensorflow/python/client/session.py", line 1451

KeyboardInterrupt: 